In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import h3

from shapely.geometry import Point
from shapely.geometry import Polygon

import contextily as ctx

#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
filename_DD = '../data/nextbike/trips_Dresden 2025-03-14_19-08-34.pkl'
filename_FB = '../data/nextbike/trips_Freiburg 2025-03-14_19-08-34.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [ ]:
def plot_df(df, column=None, ax=None, add_basemap=True):
    "Plot based on the `geometry` column of a GeoPandas dataframe"
    df = df.copy()
    df = df.to_crs(epsg=3857)  # web mercator

    if ax is None:
        _, ax = plt.subplots(figsize=(8,8))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    df.plot(
        ax=ax,
        alpha=0.25, edgecolor='k',
        column=column, categorical=True,
        legend=True, legend_kwds={'loc': 'upper left'},
    )
    if add_basemap:
        ctx.add_basemap(ax, crs=df.crs, source=ctx.providers.CartoDB.Positron)

In [ ]:
def plot_shape(shape, ax=None, add_basemap=True):
    df = gpd.GeoDataFrame({'geometry': [shape]}, crs='EPSG:4326')
    plot_df(df, ax=ax, add_basemap=add_basemap)

In [ ]:
def plot_cell(cell, ax=None):
    shape = h3.cells_to_h3shape([cell])
    plot_shape(shape, ax=ax)

In [ ]:
def plot_cells(cells, ax=None):
    fig, ax = plt.subplots(figsize=(8,8))
    shape = h3.cells_to_h3shape(cells)
    plot_shape(shape, ax=ax, add_basemap=True)
    
    for single_cell in cells:
        single_shape = h3.cells_to_h3shape([single_cell])
        # gdf = gpd.GeoDataFrame({'geometry': [single_shape]}, crs='EPSG:4326')
        # gdf = gdf.to_crs(epsg=3857)
        # gdf.plot(ax=ax, alpha=0.5, edgecolor='k')
        plot_shape(single_shape, ax=ax, add_basemap=False)


In [ ]:
resolution = 7

In [ ]:
df_DD['hex_id_return'] = df_DD.apply(lambda row: h3.latlng_to_cell(row['lat_return'], row['lng_return'], resolution), axis=1)

In [ ]:
df_FB['hex_id_return'] = df_FB.apply(lambda row: h3.latlng_to_cell(row['lat_return'], row['lng_return'], resolution), axis=1)

In [ ]:
df_DD['hex_id_rent'] = df_DD.apply(lambda row: h3.latlng_to_cell(row['lat_rent'], row['lng_rent'], resolution), axis=1)

In [ ]:
df_FB['hex_id_rent'] = df_FB.apply(lambda row: h3.latlng_to_cell(row['lat_rent'], row['lng_rent'], resolution), axis=1)

In [ ]:
len(df_FB['hex_id_return'].unique())

In [ ]:
len(df_FB['hex_id_rent'].unique())

In [ ]:
df_FB['hex_id_rent'].value_counts()

In [ ]:
df_FB['hex_id_return'].value_counts()

In [ ]:
df_DD['hex_id_return'].value_counts()

In [ ]:
df_FB['hex_id_return'].value_counts().plot(kind='bar', figsize=(20,10))

In [ ]:
df_DD['hex_id_return'].value_counts().plot(kind='bar', figsize=(20,10))

In [ ]:
len(df_DD['hex_id_return'].unique())

In [ ]:
len(df_FB['hex_id_return'].unique())

In [ ]:
# def h3_to_polygon(hex_id):
#     boundary = h3.h3_to_geo_boundary(hex_id, geo_json=True)
#     return Polygon(boundary)

# # Create a GeoDataFrame of hexagons
# hex_gdf = gpd.GeoDataFrame(df_DD['hex_id'].drop_duplicates(), 
#                            geometry=df_DD['hex_id'].drop_duplicates().apply(lambda x: Point(h3.cell_to_latlng(x)[1], h3.cell_to_latlng(x)[0])), 
#                            crs="EPSG:4326")

# # Plot hexagons
# fig, ax = plt.subplots(figsize=(8, 8))
# hex_gdf.plot(ax=ax, edgecolor="black", facecolor="lightblue", alpha=0.6)
# # plt.scatter(df_DD["lng_return"], df_DD["lat_return"], color="red", marker="o", label="Data Points")  # Plot points
# plt.legend()
# plt.title("H3 Hexagons with Data Points")
# plt.show()

In [ ]:
mycell = h3.latlng_to_cell(51.050409, 13.737262, resolution)

In [ ]:
mycell

In [ ]:
h3.cell_to_boundary(mycell)

In [ ]:
df_tmp = df_DD
geometry = [Point(xy) for xy in zip(df_tmp.lng_return, df_tmp.lat_return)]
gdf = gpd.GeoDataFrame(df_tmp, geometry=geometry, crs="EPSG:4326") 

In [ ]:
plot_cell(mycell)

In [ ]:
plot_cells(df_FB['hex_id_return'].unique())


In [ ]:
plot_cells(df_FB['hex_id_rent'].unique())


In [ ]:
plot_cells(df_DD['hex_id_return'].unique())


In [ ]:
plot_cells(df_DD['hex_id_rent'].unique())


In [ ]:
# def h3_to_polygon(hex_id):
#     boundary = h3.cell_to_boundary(hex_id)  # Correct function
#     return Polygon(boundary)

# # Example: List of hex IDs
# hex_cells = df_DD.hex_id.unique().tolist()  # Replace with your hex IDs

# # Create GeoDataFrame
# hex_gdf = gpd.GeoDataFrame(geometry=[h3_to_polygon(h) for h in hex_cells], crs="EPSG:4326")

# # Plot hexagons
# fig, ax = plt.subplots(figsize=(8, 8))
# hex_gdf.plot(ax=ax, edgecolor="black", facecolor="lightblue", alpha=0.6)
# plt.show()

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df_DD.to_pickle(f'../data/nextbike/trips_Dresden with hexagons {time}.pkl')
df_FB.to_pickle(f'../data/nextbike/trips_Freiburg with hexagons {time}.pkl')